# Imports

In [25]:
import gmsh
import numpy as np
from scipy.constants import mu_0, epsilon_0

# Mesh Parameters

In [26]:
FINITE_ELEMENT = ("Tetrahedron", 1)
BOUNDARY = [{'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}]
MATERIAL = [{'tag': 301, 'name': 'free_space', 'ur': 1, 'er': 1}]
element, order = FINITE_ELEMENT

# Geometrical Data

In [27]:
OMEGA = 2 * np.pi * 3E8
K0 = OMEGA * np.sqrt(mu_0 * epsilon_0)
WAVELENGTH = 2 * np.pi / K0

h = WAVELENGTH*2    # Tamanho do elemento finito
L = WAVELENGTH/2    # Lado do cubo externo
ra = WAVELENGTH/2   # Raio da esfera
x0 = WAVELENGTH     # Lado do cubo interno

# Gmsh Initialize

In [28]:
gmsh.initialize()
gmsh.model.add("cavity_pml")
factory = gmsh.model.occ

## Define domain

In [29]:
# PML cúbicas Inferior Esquerda
region_a = factory.addBox(-x0-L, -x0-L, +x0, L, L, L)
region_b = factory.addBox(-x0-L, -x0-L, -x0-L, L, L, L)

# PML cúbicas Inferior Direita
region_c = factory.addBox(x0, -x0-L, +x0, L, L, L)
region_d = factory.addBox(x0, -x0-L, -x0-L, L, L, L)

# PML cúbicas Superior Direita
region_e = factory.addBox(x0, x0, +x0, L, L, L)
region_f = factory.addBox(x0, x0, -x0-L, L, L, L)

# PML cúbicas Inferior Esquerda
region_g = factory.addBox(-x0-L, x0, +x0, L, L, L)
region_h = factory.addBox(-x0-L, x0, -x0-L, L, L, L)

# PML longitudinais Inferiores
region_1 = factory.addBox(x0, -x0-L, -x0, L, L, 2*x0)
region_2 = factory.addBox(-x0-L, -x0-L, -x0, L, L, 2*x0)

# PML longitudinais Superiores
region_3 = factory.addBox(x0, x0, -x0, L, L, 2*x0)
region_4 = factory.addBox(-x0-L, x0, -x0, L, L, 2*x0)

# PML horizontais Inferiores
region_5 = factory.addBox(-x0, -x0-L, x0, 2*x0, L, L)
region_6 = factory.addBox(-x0, -x0-L, -x0-L, 2*x0, L, L)

# PML horizontais superiores
region_7 = factory.addBox(-x0, x0, x0, 2*x0, L, L)
region_8 = factory.addBox(-x0, x0, -x0-L, 2*x0, L, L)

# PML verticais direita
region_9 = factory.addBox(x0, -x0, x0, L, 2*x0, L)
region_10 = factory.addBox(x0, -x0, -x0-L, L, 2*x0, L)

# PML verticais esquerda
region_11 = factory.addBox(-x0-L, -x0, x0, L, 2*x0, L)
region_12 = factory.addBox(-x0-L, -x0, -x0-L, L, 2*x0, L)

# PML Piso e Teto
region_13 = factory.addBox(-x0, -x0-L, -x0, 2*x0, L, 2*x0)
region_14 = factory.addBox(-x0, x0, -x0, 2*x0, L, 2*x0)

# PML Parede Direita e Esquerda
region_15 = factory.addBox(x0, -x0, -x0, L, 2*x0, 2*x0)
region_16 = factory.addBox(-x0-L, -x0, -x0, L, 2*x0, 2*x0)

# PML Parede Frontal e Traseira
region_17 = factory.addBox(-x0, -x0, x0, 2*x0, 2*x0, L)
region_18 = factory.addBox(-x0, -x0, -x0-L, 2*x0, 2*x0, L)

# Criar região do espaço livre (cubo interno)
region_fs = factory.addBox(-x0, -x0, -x0, 2*x0, 2*x0, 2*x0)

# Criar região do espalhador (esfera)
# sphere = factory.addSphere(0, 0, 0, ra)

In [30]:
# Regiões PML em termos de coordenadas
PML_xyz = [region_a, region_b, region_c, region_d, region_e, region_f, region_g, region_h]
PML_xy = [region_1, region_2, region_3, region_4]
PML_yz = [region_5, region_6, region_7, region_8]
PML_xz = [region_9, region_10, region_11, region_12]
PML_y = [region_13, region_14]
PML_x = [region_15, region_16]
PML_z = [region_17, region_18]

# Fragmentar todas as regiões para garantir interfaces conformais
PML_list = PML_xy + PML_yz + PML_xz + PML_y + PML_x + PML_z + PML_xyz
objectDimTags = [(3, region_fs)] + [(3, item) for item in PML_list]
outDimTags, _ = factory.fragment(objectDimTags, objectDimTags)

# Subtrair a esfera do espaço livre
# outDimTags_omega_s, _ = factory.cut([(3, region_fs)], [(3, sphere)], removeTool=True)                   

# Sincronizar
factory.synchronize()

## Mesh Size 

In [31]:
gmsh.option.setNumber("Mesh.MeshSizeMax", h)
gmsh.option.setNumber("Mesh.MeshSizeMin", h)
gmsh.model.mesh.generate(dim=3)
gmsh.model.mesh.setOrder(order)

## 1. Mesh Elements by Dimension
### Dimension 0 (1-node point)

In [32]:
elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=0)
print(f"elemTypes: {elementTypes}")
print(f"elemTags: {elementTags}")
print(f"Total elements: {[len(elementTag) for elementTag in elementTags]}")
print(f"elemNodeTags: {nodeTags}")
print(f"Total element nodes: {[len(nodeTag) for nodeTag in nodeTags]}")

elemTypes: [15]
elemTags: [array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64], dtype=uint64)]
Total elements: [64]
elemNodeTags: [array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64], dtype=uint64)]
Total element nodes: [64]


### Dimension 1 (2-node line)

In [33]:
elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=1)
print(f"elemTypes: {elementTypes}")
print(f"elemTags: {elementTags}")
print(f"Total elements: {[len(elementTag) for elementTag in elementTags]}")
print(f"elemNodeTags: {nodeTags}")
print(f"Total element nodes: {[len(nodeTag) for nodeTag in nodeTags]}")

elemTypes: [1]
elemTags: [array([ 65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
       208], dtype=uint64)]
Total elements: [144]
elemNodeTags: [array([ 1,  2,  2,  3,  4,  3,  1,  4,  5,  6,  6,  7,  8,  7,  5,  8,  1,
        5,  2,  6,  4,  8,  3,  7,  9, 10,

### Dimension 2 (3-node triangle)

In [34]:
elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=2)
print(f"elemTypes: {elementTypes}")
print(f"elemTags: {elementTags}")
print(f"Total elements: {[len(elementTag) for elementTag in elementTags]}")
print(f"elemNodeTags: {nodeTags}")
print(f"Total element nodes: {[len(nodeTag) for nodeTag in nodeTags]}")

elemTypes: [2]
elemTags: [array([209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,
       222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234,
       235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247,
       248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260,
       261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273,
       274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286,
       287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299,
       300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312,
       313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325,
       326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338,
       339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351,
       352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364,
       365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 3

### Dimension 3 (4-node tetrahedron)

In [35]:
elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=3)
print(f"elemTypes: {elementTypes}")
print(f"elemTags: {elementTags}")
print(f"Total elements: {[len(elementTag) for elementTag in elementTags]}")
print(f"elemNodeTags: {nodeTags}")
print(f"Total element nodes: {[len(nodeTag) for nodeTag in nodeTags]}")

elemTypes: [4]
elemTags: [array([545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557,
       558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583,
       584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596,
       597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609,
       610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622,
       623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635,
       636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648,
       649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661,
       662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674,
       675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687,
       688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700,
       701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713,
       714, 715, 716, 717, 718, 719, 7

## 2. Geometrical Entities by Dimension
### Dimension 0 (points)

In [36]:
dimTags = gmsh.model.getEntities(0)
print(f"Total 0-D entities: {len(dimTags)}")
print(f"dimTags: {dimTags[:5]} ...")

Total 0-D entities: 64
dimTags: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)] ...


## Dimension 1 (lines)

In [37]:
dimTags = gmsh.model.getEntities(1)
print(f"Total 1-D entities: {len(dimTags)}")
print(f"dimTags: {dimTags[:5]} ...")

Total 1-D entities: 144
dimTags: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)] ...


## Dimension 2 (surfaces)

In [38]:
dimTags = gmsh.model.getEntities(2)
print(f"Total 2-D entities: {len(dimTags)}")
print(f"dimTags: {dimTags[:5]} ...")

Total 2-D entities: 108
dimTags: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)] ...


## 1. _Edge mapping_

In [39]:
gmsh.model.mesh.createEdges()
edgeTags, edgeNodes = gmsh.model.mesh.getAllEdges()
print(f"Total number of edges: {len(edgeTags)}")
print(f"Total number of edgeNodes: {len(edgeNodes)}")

Total number of edges: 663
Total number of edgeNodes: 1326


In [40]:
# Criar edge_mapping
edge_mapping = {tuple(sorted([edgeNodes[2*i], edgeNodes[2*i + 1]])): tag
                 for i, tag in enumerate(sorted(edgeTags))}

In [41]:
for key in list(edge_mapping.keys())[:10]:
    print(f"{key}: {edge_mapping[key]}")

(67, 70): 1
(65, 67): 2
(67, 69): 3
(66, 70): 4
(68, 69): 5
(66, 69): 6
(66, 68): 7
(40, 69): 8
(7, 99): 9
(3, 99): 10


## 2. _Face mapping_

In [42]:
gmsh.model.mesh.createFaces()
faceTags, faceNodes = gmsh.model.mesh.getAllFaces(3)
print(f"Total number of faces: {len(faceTags)}")
print(f"Total number of faceNodes: {len(faceNodes)}")

Total number of faces: 988
Total number of faceNodes: 2964


In [43]:
# Criar face_mapping
face_mapping = {tuple(sorted([faceNodes[3*i], faceNodes[3*i + 1], faceNodes[3*i + 2]])): tag
                 for i, tag in enumerate(sorted(faceTags))}

In [44]:
for key in list(face_mapping.keys())[:10]:
    print(f"{key}: {face_mapping[key]}")

(5, 66, 67): 1
(7, 68, 70): 2
(8, 66, 68): 3
(1, 65, 67): 4
(6, 66, 70): 5
(4, 65, 68): 6
(3, 65, 68): 7
(5, 67, 69): 8
(1, 67, 69): 9
(5, 66, 69): 10


## 3. Applying _Physical Groups_
### 3.1 Surfaces

In [45]:
# Obter os contornos (superfícies, dim=2) do volume
BoundaryDimTags = gmsh.model.getBoundary([(3, rectangular_cavity)], oriented=True, recursive=False)
print(f"BoundaryDimTags: {BoundaryDimTags}")

NameError: name 'rectangular_cavity' is not defined

In [ ]:
# Exibir os TAGs das superfícies associadas a cada contorno
BoundaryTags = [Dimtags[1] for Dimtags in BoundaryDimTags]

# Definindo as superfícies de contorno de Dirichlet (dim=2)
gmsh.model.addPhysicalGroup(dim=2, tags=BoundaryTags, tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])

# Adicionar grupos físicos para Dim=3 (volume)
gmsh.model.addPhysicalGroup(dim=3, tags=[rectangular_cavity], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])

## 5. Geometrical Entities by Dimension
dimTags (vector of pairs of integers)
### Dimension 0 (Points)

In [ ]:
gmsh.model.getEntities(dim=0)

### Dimension 1 (Edges)

gmsh.model.getEntities(dim=1)

### Dimension 2 (Surfaces)

In [ ]:
gmsh.model.getEntities(dim=2)

### Dimension 3 (Regions)

In [ ]:
gmsh.model.getEntities(dim=3)

## 6. ``mesh_data['cell']``: Material Physical Groups (dim=3)

In [ ]:
cell_data = {}
for material in MATERIAL:
    # Obter as entidades físicas (grupo físico) associadas ao material
    MaterialEntitiesTags = gmsh.model.getEntitiesForPhysicalGroup(dim=3, tag=material['tag'])
    print(f"Entities of {material['name']} with dim3: {MaterialEntitiesTags}")
    
    for EntityTag in MaterialEntitiesTags:
        elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=3, tag=EntityTag)
        print(f"mesh_elements of entity (dim, tag) = (3, {EntityTag}): elemTags = {elementTags}")
        print(f"mesh_elements of entity (dim, tag) = (3, {EntityTag}): nodeTags = {nodeTags}")
        
        for elemType, elemTag, elemNode in zip(elementTypes, elementTags, nodeTags):            
            # Obter as propriedades do elemento
            _, _, _, nodes_per_element, _, _ = gmsh.model.mesh.getElementProperties(elemType)

            # Número de elementos
            N_tet = len(elemNode) // nodes_per_element

            # Criar dicionário associando cada elemento à sua conectividade de nós
            for i, Tag in enumerate(elemTag):                
                # Obter a conectividade do elemento
                # conn_node = list(sorted(elemNodes[0][4 * i : 4 * (i + 1)])) 
                conn_node = elemNode[nodes_per_element * i: nodes_per_element * (i + 1)].tolist()
                conn_std = sorted(conn_node)

                # Tetrahedron element
                if nodes_per_element == 4:
                    conn_edge = [
                        edge_mapping[(conn_std[0], conn_std[1])],  # e1: 1 -> 2
                        edge_mapping[(conn_std[0], conn_std[2])],  # e2: 1 -> 3
                        edge_mapping[(conn_std[0], conn_std[3])],  # e3: 1 -> 4
                        edge_mapping[(conn_std[1], conn_std[2])],  # e4: 2 -> 3
                        edge_mapping[(conn_std[1], conn_std[3])],  # e5: 2 -> 4
                        edge_mapping[(conn_std[2], conn_std[3])]   # e6: 3 -> 4
                    ]

                    # Conectividade de faces (cada face tem 3 nós)
                    conn_face = [
                        sorted([conn_node[0], conn_node[1], conn_node[2]]),  # Face 1
                        sorted([conn_node[0], conn_node[1], conn_node[3]]),  # Face 2
                        sorted([conn_node[0], conn_node[2], conn_node[3]]),  # Face 3
                        sorted([conn_node[1], conn_node[2], conn_node[3]])   # Face 4
                    ]
                
                # Adicionar ao dicionário de células
                cell_data[Tag] = {
                    'tag': Tag,
                    'conn': conn_node,
                    'conn_sorted': conn_std,
                    'conn_edge': conn_edge,
                    'conn_face': conn_face,
                    # 'geo': {'centroid': None, 'dim': None},
                    # 'contour': {'type': None, 'conn_contour': None},
                    'material': material['tag']}

In [ ]:
# Adiciona nova chave sequencial ao cell_data
cell_data = {i+1: cell_data[Tag] for i, Tag in enumerate(cell_data)}; cell_data

## 7. `mesh_data['boundary']`: Boundary Physical Groups (dim=2)

In [ ]:
boundary_data = {}
for bc in BOUNDARY:
    # Obter as entidades físicas (grupo físico) associadas ao contorno
    BoundaryEntitiesTags = gmsh.model.getEntitiesForPhysicalGroup(dim=2, tag=bc['tag'])
    print(f"Entities of {bc['name']} with dim2: {BoundaryEntitiesTags}")
    
    for EntityTag in BoundaryEntitiesTags:
        elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=2, tag=EntityTag)
        print(f"mesh_elements of entity (dim, tag) = (2, {EntityTag}): elemTags = {elementTags}")
        print(f"mesh_elements of entity (dim, tag) = (2, {EntityTag}): nodeTags = {nodeTags}")
        
        for elemType, elemTag, elemNode in zip(elementTypes, elementTags, nodeTags):   
            # Obter as propriedades do elemento
            _, _, _, nodes_per_element, _, _ = gmsh.model.mesh.getElementProperties(elemType)

            # Número de elementos
            N_tri = len(elemNode) // nodes_per_element             

            # Criar dicionário associando cada elemento à sua conectividade de nós
            for i, Tag in enumerate(elemTag):    
                # Obter a conectividade do elemento            
                # conn_node = list(sorted(nodeTags[0][3 * i : 3 * (i + 1)]))
                conn_node = elemNode[nodes_per_element * i: nodes_per_element * (i + 1)].tolist()
                conn_std = sorted(conn_node)

                # Triangular element
                if nodes_per_element == 3:
                    conn_edge = [
                        edge_mapping[(conn_std[0], conn_std[1])],  # e1: 1 -> 2
                        edge_mapping[(conn_std[0], conn_std[2])],  # e2: 1 -> 3
                        edge_mapping[(conn_std[1], conn_std[2])],  # e3: 2 -> 3
                    ]

                # Dicionário de contorno
                boundary_data[Tag] = {
                    'tag': Tag,
                    'conn': conn_node,
                    'conn_sorted': conn_std,
                    'conn_edge': conn_edge,
                    'boundary': bc['tag']}

In [ ]:
# Adiciona nova chave sequencial ao boundary_data
boundary_data = {i+1: boundary_data[Tag] for i, Tag in enumerate(boundary_data)}; boundary_data

# Gmsh Finalize

In [ ]:
# Visualizar a malha no ambiente Gmsh (opcional)
gmsh.fltk.run()

# Finalizar o Gmsh
gmsh.finalize()